In [1]:
import pandas as pd
import os

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

from spacy import displacy
import edsnlp, edsnlp.pipes as eds

input_dir = "../data_clean/"
visuals_dir = "../visuals/" 

In [ ]:
df_person = pd.read_pickle(os.path.join(input_dir, 'df_person.pkl'))
df_bio = pd.read_pickle(os.path.join(input_dir, 'df_bio.pkl'))
df_note = pd.read_pickle(os.path.join(input_dir, 'df_note.pkl'))
df_visit = pd.read_pickle(os.path.join(input_dir, 'df_visit.pkl'))
df_condition = pd.read_pickle(os.path.join(input_dir, 'df_condition.pkl'))

In [7]:
nlp_smoker = edsnlp.blank("eds")
nlp_smoker.add_pipe(eds.sentences())
nlp_smoker.add_pipe(eds.normalizer())
nlp_smoker.add_pipe(eds.tobacco())
nlp_smoker.add_pipe(eds.negation())
nlp_smoker.add_pipe(eds.family())

nlp_alcool = edsnlp.blank("eds")
nlp_alcool.add_pipe(eds.sentences())
nlp_alcool.add_pipe(eds.normalizer()) 
nlp_alcool.add_pipe(eds.alcohol())
nlp_alcool.add_pipe(eds.negation())
nlp_alcool.add_pipe(eds.family())

2025-06-06 09:16:48.642 | WARNING  | edsnlp.pipes.qualifiers.family.family:__init__:171 - You have requested that the pipeline use annotations provided by the `section` pipeline, but it was not set. Skipping that step.
2025-06-06 09:16:48.718 | WARNING  | edsnlp.pipes.qualifiers.family.family:__init__:171 - You have requested that the pipeline use annotations provided by the `section` pipeline, but it was not set. Skipping that step.


In [8]:
smokers, total = 0, 0
for i in range(len(df_note)):
    note = df_note.iloc[i]
    text = note['note_text']
    doc = nlp_smoker(text)

    filtered_ents_for_display = [ent for ent in doc.ents if not ent._.family]
    filtered_ents_for_display = [ent for ent in filtered_ents_for_display if not ent._.negation]

    visit_id_from_sampled_note = note['visit_occurrence_id']
    matching_visit_rows = df_visit[df_visit['visit_occurrence_id'] == visit_id_from_sampled_note]

    current_note_id_scalar = note['note_id']
    person_id = matching_visit_rows['person_id'].iloc[0]
   
    if matching_visit_rows.empty:
        raise ValueError(f"No matching visit found for visit_occurrence_id: {visit_id_from_sampled_note}")
    total += 1
    if filtered_ents_for_display: 
        smokers += 1
        
        df_person.loc[df_person['person_id'] == person_id, 'fumeur'] = True
    else:
        df_person.loc[df_person['person_id'] == person_id, 'fumeur'] = False

print(f"Total notes processed: {total}")
print(f"Total smokers identified: {smokers}")

Total notes processed: 958
Total smokers identified: 90


In [ ]:
alcohol, total = 0, 0

for i in range(len(df_note)):
    note = df_note.iloc[i]
    text = note['note_text']
    doc = nlp_alcool(text)

    filtered_ents_for_display = [ent for ent in doc.ents if not ent._.family]
    filtered_ents_for_display = [ent for ent in filtered_ents_for_display if not ent._.negation]

    visit_id_from_sampled_note = note['visit_occurrence_id']
    matching_visit_rows = df_visit[df_visit['visit_occurrence_id'] == visit_id_from_sampled_note]

    current_note_id_scalar = note['note_id']
    person_id = matching_visit_rows['person_id'].iloc[0]
   
    total += 1
    if filtered_ents_for_display:
        alcohol += 1
        df_person.loc[df_person['person_id'] == person_id, 'alcool'] = True
    else:
        df_person.loc[df_person['person_id'] == person_id, 'alcool'] = False
        
print(df_person.head())
print(f"Total notes processed: {total}")
print(f"Total alcohol consumers identified: {alcohol}")

pd.to_pickle(df_person, os.path.join(input_dir, 'df_person.pkl'))


2025-06-06 09:14:31.004 | WARNING  | edsnlp.pipes.qualifiers.family.family:__init__:171 - You have requested that the pipeline use annotations provided by the `section` pipeline, but it was not set. Skipping that step.


  birth_datetime death_datetime gender_source_value   person_id  \
0     1961-07-09     2022-02-20              female  80006137.0   
1     1951-12-31     2024-07-28              female  80021799.0   
2     1959-11-16            NaT              female  80034258.0   
3     1975-01-23            NaT              female  80057927.0   
4     1951-10-29            NaT              female  80071532.0   

   unique_person_id  prob fumeur  alcool  
0        80006137.0   NaN  False   False  
1        80021799.0   NaN  False   False  
2        80034258.0   NaN  False   False  
3        89552692.0   1.0    NaN   False  
4        80071532.0   NaN  False   False  
Total notes processed: 958
Total alcohol consumers identified: 22
